In [11]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
# from xgboost import XGBRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


data_train = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_nonscaled.parquet')

data_test = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_nonscaled.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

# List of columns to drop
columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:])
# columns_to_drop = ['trading_days_till_exp']


# Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
test_data = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_test = data_test[data_train.columns]

data_trains = data_train.copy()

data_train = data_trains[data_trains['date'] < '2020-01-01']
data_validate = data_trains[data_trains['date'] >= '2020-01-01']

# (data_test)


In [12]:
# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare validation data for Call and Put options
data_validate_c = data_validate[data_validate['cp_flag'] == 'C']
data_validate_p = data_validate[data_validate['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

data_test_c

,cp_flag,Ticker,date,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,BIDLO,ASKHI,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
0,C,AAPL,2021-01-04,-0.475,0.433412,0.019841,0.337114,0.325650,133.39999,135.99001,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
1,C,AAPL,2021-01-04,-0.345,0.428775,0.019841,0.332640,0.312515,133.39999,135.99001,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
2,C,AAPL,2021-01-04,-0.216,0.426358,0.019841,0.330864,0.308907,133.39999,135.99001,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
3,C,AAPL,2021-01-04,-0.088,0.422649,0.019841,0.329431,0.306933,133.39999,135.99001,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
4,C,AAPL,2021-01-04,0.039,0.421479,0.019841,0.328318,0.305940,133.39999,135.99001,...,672.012903,3.036452,0.93,0.1074,0.1400,21.29,22.49,20.57,21.20,1.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36347,C,TSLA,2021-12-30,1.556,0.742669,0.007937,0.718058,0.773147,1078.42004,1118.99988,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
36348,C,TSLA,2021-12-30,1.658,0.764381,0.007937,0.735985,0.789361,1078.42004,1118.99988,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
36349,C,TSLA,2021-12-30,1.760,0.786317,0.007937,0.754168,0.807648,1078.42004,1118.99988,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04
36350,C,TSLA,2021-12-30,1.861,0.798978,0.007937,0.773210,0.821686,1078.42004,1118.99988,...,632.874194,-0.206774,1.55,0.4878,0.8182,14.36,14.60,12.56,12.85,2.04


In [16]:
import time
import itertools
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):
    """
    Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
    retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test).
    
    Parameters:
    train_data (pd.DataFrame): The training dataset (Year 1).
    validate_data (pd.DataFrame): The validation dataset (Year 2).
    test_data (pd.DataFrame): The testing dataset (Year 3).
    option_type (str): Call or Put option type for labeling the print output.
    verbose (bool): If True, prints progress information for hyperparameter tuning.
    """
    # Prepare the train, validation, and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
    y_train = train_data['impl_volatility']  # Training target (Year 1)

    X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
    y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
    y_test = test_data['impl_volatility']  # Test target (Year 3)


    # Define the hyperparameter grid
    # param_grid = {
    #     'max_depth': [5, 7, 9, 11],                 
    #     'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800],             
    #     'learning_rate': [0.005, 0.01, 0.05, 0.1],             
    # }
    
    param_grid = {'learning_rate': [0.01], 'max_depth': [5], 'n_estimators': [500]}
    # param_grid = {'learning_rate': [0.01], 'max_depth': [3], 'n_estimators': [700]}
    # param_grid = {'learning_rate': [0.01], 'max_depth': [11], 'n_estimators': [700]}

    # Generate all combinations of hyperparameters
    param_combinations = list(itertools.product(
        param_grid['max_depth'], 
        param_grid['n_estimators'], 
        param_grid['learning_rate']
    ))

    total_combinations = len(param_combinations)
    
    # Initialize variables to store the best model and best score
    best_rmse_val = np.inf
    best_params = None
    best_xgb_model = None

    print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
    # Iterate over all hyperparameter combinations with progress tracking
    for i, (max_depth, n_estimators, learning_rate) in enumerate(param_combinations):
        start_time = time.time()
        
        # Initialize the XGBRegressor with the current set of hyperparameters
        xgb_model = XGBRegressor(
            max_depth=max_depth, 
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            objective='reg:squarederror',
            n_jobs=-1
        )

        # Train the model on the training data (Year 1)
        xgb_model.fit(X_train, y_train)

        # Validate the model on the validation data (Year 2)
        y_val_pred = xgb_model.predict(X_validate)
        rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

        # Track the best performing hyperparameters based on validation RMSE
        if rmse_val < best_rmse_val:
            best_rmse_val = rmse_val
            best_params = {
                'max_depth': max_depth,
                'n_estimators': n_estimators,
                'learning_rate': learning_rate
            }
            best_xgb_model = xgb_model

        # Verbose output to track progress
        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
            print(f"Current RMSE (Validation): {rmse_val:.4f}")
            print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
    print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
    # Retrain the model on combined Year 1 (train) and Year 2 (validation)
    print("Retraining the model on Year 1 and Year 2 combined...")
    X_combined = pd.concat([X_train, X_validate])
    y_combined = pd.concat([y_train, y_validate])
    best_xgb_model.fit(X_combined, y_combined)

    # In-sample (combined Year 1 + Year 2) predictions
    y_combined_pred = best_xgb_model.predict(X_combined)

    # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
    rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
    r2_combined = r2_score(y_combined, y_combined_pred)
    
    print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
    print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
    print(f"R² (Training + Validation): {r2_combined:.4f}")

    # After retraining, evaluate performance on the test data (Year 3)
    y_test_pred = best_xgb_model.predict(X_test)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the final results
    print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


Running manual hyperparameter tuning for Call Options...
Combination 1/1 completed in 11.68 seconds.
Current RMSE (Validation): 0.2999
Best RMSE so far: 0.2999

Best Parameters for Call Options: {'max_depth': 11, 'n_estimators': 700, 'learning_rate': 0.01}
Retraining the model on Year 1 and Year 2 combined...

In-Sample Performance for Call Options (Year 1 + Year 2):
RMSE (Training + Validation): 0.0222
R² (Training + Validation): 0.9963

Performance on Test Data (Year 3) for Call Options:
Out-of-Sample RMSE (Test): 0.1427
Out-of-Sample R² (Test): 0.6420
Running manual hyperparameter tuning for Put Options...
Combination 1/1 completed in 11.09 seconds.
Current RMSE (Validation): 0.2973
Best RMSE so far: 0.2973

Best Parameters for Put Options: {'max_depth': 11, 'n_estimators': 700, 'learning_rate': 0.01}
Retraining the model on Year 1 and Year 2 combined...

In-Sample Performance for Put Options (Year 1 + Year 2):
RMSE (Training + Validation): 0.0231
R² (Training + Validation): 0.9965


In [3]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV
# from xgboost import XGBRegressor

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare validation data for Call and Put options
# data_validate_c = data_validate[data_validate['cp_flag'] == 'C']
# data_validate_p = data_validate[data_validate['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data_with_gridsearch(train_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target

#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
#     y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target

#     param_grid = {
#     'max_depth': [7, 9, 11, 13, 15],                 
#     'n_estimators': [100, 200, 300, 400, 500, 600],             # Increase number of trees for better generalization
#     'learning_rate': [0.001, 0.01, 0.1],             # Ensure minimum leaf size is larger to reduce overfitting
#     }   

#     # Initialize the XGBRegressor
#     XGBmodel = XGBRegressor()

#     # # Initialize GridSearchCV
#     grid_search = GridSearchCV(estimator=XGBmodel, param_grid=(param_grid), 
#                                cv=5, n_jobs=-1, verbose=2)

#     # # Train the model using GridSearchCV
#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Get the best parameters and the best estimator
#     best_XGB_model = grid_search.best_estimator_
#     # best_XGB_params = {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500}
#     # best_XGB_model = XGBRegressor(**best_XGB_params)
#     print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

#     # In-sample (training set) predictions
#     y_train_pred = best_XGB_model.predict(X_train)

#     # Out-of-sample (test set) predictions
#     y_test_pred = best_XGB_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


# New improved set to get reduce overfit

In [4]:
# import numpy as np
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.model_selection import GridSearchCV

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data_with_gridsearch(train_data, test_data, option_type):
#     """
#     Prepare the data, train XGBoost using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target

#     # Define parameter grid for GridSearchCV
#     param_grid = {
#         'max_depth': [3, 5, 7],                  
#         'n_estimators': [100, 200, 300],    
#         'learning_rate': [0.001, 0.01, 0.1],    
#         'subsample': [0.7, 0.8, 0.9],           
#         'colsample_bytree': [0.7, 0.8, 0.9],    
#         'reg_alpha': [0, 0.1, 0.5],             
#         'reg_lambda': [1, 1.5, 2],              
#     }   

#     # Initialize the XGBRegressor
#     XGBmodel = XGBRegressor()

#     # Perform GridSearchCV to find the best hyperparameters
#     grid_search = GridSearchCV(estimator=XGBmodel, param_grid=param_grid, 
#                                cv=5, n_jobs=-1, verbose=2)

#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Get the best parameters and use them for the final model
#     best_params = grid_search.best_params_
#     print(f"\nBest Parameters for {option_type} Options: {best_params}")

#     # Initialize the best model with early stopping manually
#     best_XGB_model = XGBRegressor(**best_params)

#     print(f"\nTraining the model with early stopping for {option_type} Options...")
#     best_XGB_model.fit(X_train, y_train, 
#                        eval_set=[(X_test, y_test)], 
#                        early_stopping_rounds=10, 
#                        verbose=True)

#     # In-sample (training set) predictions
#     y_train_pred = best_XGB_model.predict(X_train)

#     # Out-of-sample (test set) predictions
#     y_test_pred = best_XGB_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


In [5]:
print("hello")

hello


In [9]:
import time
import itertools
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd

def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):
    """
    Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
    retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test).
    
    Parameters:
    train_data (pd.DataFrame): The training dataset (Year 1).
    validate_data (pd.DataFrame): The validation dataset (Year 2).
    test_data (pd.DataFrame): The testing dataset (Year 3).
    option_type (str): Call or Put option type for labeling the print output.
    verbose (bool): If True, prints progress information for hyperparameter tuning.
    """
    # Prepare the train, validation, and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
    y_train = train_data['impl_volatility']  # Training target (Year 1)

    X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
    y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
    y_test = test_data['impl_volatility']  # Test target (Year 3)

    # Define the hyperparameter grid
    # param_grid = {
    #     'max_depth': [3, 5, 7],                  
    #     'n_estimators': [100, 200, 300, 500],    
    #     'learning_rate': [0.001, 0.01, 0.1],    
    #     'subsample': [0.7, 0.8, 0.9],           
    #     'colsample_bytree': [0.7, 0.8, 0.9],    
    #     'reg_alpha': [0, 0.1, 0.5],             
    #     'reg_lambda': [1, 1.5, 2],              
    # }

    # # Best params put
    # param_grid = {'max_depth': [3], 'n_estimators': [100], 'learning_rate': [0.1], 'subsample': [0.7], 'colsample_bytree': [0.9], 'reg_alpha': [0], 'reg_lambda': [1.5]}

    # Best params call
    param_grid = {'max_depth': [3], 'n_estimators': [300], 'learning_rate': [0.01], 'subsample': [0.8], 'colsample_bytree': [0.9], 'reg_alpha': [0.5], 'reg_lambda': [1.5]}
    
    # Generate all combinations of hyperparameters
    param_combinations = list(itertools.product(
        param_grid['max_depth'], 
        param_grid['n_estimators'], 
        param_grid['learning_rate'],
        param_grid['subsample'],
        param_grid['colsample_bytree'],
        param_grid['reg_alpha'],
        param_grid['reg_lambda']
        
    ))

    total_combinations = len(param_combinations)
    
    # Initialize variables to store the best model and best score
    best_rmse_val = np.inf
    best_params = None
    best_xgb_model = None

    print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
    # Iterate over all hyperparameter combinations with progress tracking
    for i, (max_depth, n_estimators, learning_rate, subsample, colsample_bytree, reg_alpha, reg_lambda) in enumerate(param_combinations):
        start_time = time.time()
        
        # Initialize the XGBRegressor with the current set of hyperparameters
        xgb_model = XGBRegressor(
            max_depth=max_depth, 
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            objective='reg:squarederror',
            n_jobs=-1
        )

        # Train the model on the training data (Year 1)
        xgb_model.fit(X_train, y_train)

        # Validate the model on the validation data (Year 2)
        y_val_pred = xgb_model.predict(X_validate)
        rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

        # Track the best performing hyperparameters based on validation RMSE
        if rmse_val < best_rmse_val:
            best_rmse_val = rmse_val
            best_params = {
                'max_depth': max_depth,
                'n_estimators': n_estimators,
                'learning_rate': learning_rate,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'reg_alpha': reg_alpha,
                'reg_lambda': reg_lambda
            }
            best_xgb_model = xgb_model

        # Verbose output to track progress
        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
            print(f"Current RMSE (Validation): {rmse_val:.4f}")
            print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
    print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
    # Retrain the model on combined Year 1 (train) and Year 2 (validation)
    print("Retraining the model on Year 1 and Year 2 combined...")
    X_combined = pd.concat([X_train, X_validate])
    y_combined = pd.concat([y_train, y_validate])
    best_xgb_model.fit(X_combined, y_combined)

    # In-sample (combined Year 1 + Year 2) predictions
    y_combined_pred = best_xgb_model.predict(X_combined)

    # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
    rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
    r2_combined = r2_score(y_combined, y_combined_pred)
    
    print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
    print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
    print(f"R² (Training + Validation): {r2_combined:.4f}")

    # After retraining, evaluate performance on the test data (Year 3)
    y_test_pred = best_xgb_model.predict(X_test)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the final results
    print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


Running manual hyperparameter tuning for Call Options...
Combination 1/2916 completed in 0.15 seconds.
Current RMSE (Validation): 0.4487
Best RMSE so far: 0.4487
Combination 2/2916 completed in 0.14 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 3/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 4/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4487
Best RMSE so far: 0.4487
Combination 5/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 6/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 7/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 8/2916 completed in 0.14 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.4487
Combination 9/2916 completed in 0.13 seconds.
Current RMSE (Validation): 0.4488
Best RMSE so far: 0.448

In [4]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train XGBoost using defined parameters, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target

    best_XGB_params = {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500}
    best_XGB_model = XGBRegressor(**best_XGB_params)

    # Fit the model to the training data
    print(f"Fitting the model for {option_type} options...")
    best_XGB_model.fit(X_train, y_train)  # This line must execute before any predictions
    
    # Sanity check to ensure model is fitted
    if not hasattr(best_XGB_model, 'get_booster'):
        raise ValueError(f"Model for {option_type} options is not fitted properly!")
    
    # In-sample (training set) predictions
    y_train_pred = best_XGB_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_XGB_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Fitting the model for Call options...

Performance for Call Options:
In-Sample RMSE (Training): 0.1085
In-Sample R² (Training): 0.9127
Out-of-Sample RMSE (Test): 0.1293
Out-of-Sample R² (Test): 0.7060
Fitting the model for Put options...

Performance for Put Options:
In-Sample RMSE (Training): 0.1087
In-Sample R² (Training): 0.9238
Out-of-Sample RMSE (Test): 0.1349
Out-of-Sample R² (Test): 0.7013
